In [ ]:
import json
import cv2
import matplotlib.pyplot as plt
import sys
import os
import numpy as np # cv2.rectangle and other operations might return numpy arrays

# --- Configuration: Adjust these paths and IDs as needed ---

# Assuming the notebook is in nutri-snap/notebooks/
# This adds the 'nutri-snap' directory to sys.path to allow imports from external_libs
current_notebook_dir = os.getcwd() # Should be /path/to/nutri-snap/notebooks
project_root = os.path.abspath(os.path.join(current_notebook_dir, '..'))

if project_root not in sys.path:
    sys.path.append(project_root)

# Now you can import from external_libs
# Make sure the path external_libs/ml_4m/fourm/utils/plotting_utils.py is correct relative to project_root
try:
    from external_libs.ml_4m.fourm.utils.plotting_utils import visualize_bboxes
except ImportError as e:
    print(f"Could not import visualize_bboxes. Make sure the file external_libs/ml_4m/fourm/utils/plotting_utils.py exists and is accessible.")
    print(f"PYTHONPATH includes: {sys.path}")
    print(f"Error details: {e}")
    # As a fallback, you might need to copy visualize_bboxes and convert_string_to_bboxes here
    # For now, we'll let it raise an error if the import fails.
    raise

# Example paths - YOU MUST MODIFY THESE to point to your actual data
# Example: to use data from 'train' split for dish_1560541706
SPLIT = 'train' # Or 'test', 'val', etc.
DISH_ID = 'dish_1560541706' # Change to your desired dish ID

# Path to the directory containing processed data (relative to the project_root)
# Default assumes nutri-snap/data/processed/
base_processed_dir_relative_to_project_root = os.path.join('data', 'processed')
absolute_base_processed_dir = os.path.join(project_root, base_processed_dir_relative_to_project_root)

rgb_image_path = os.path.join(absolute_base_processed_dir, SPLIT, 'rgb', DISH_ID, f'{DISH_ID}.png')
bbox_json_path = os.path.join(absolute_base_processed_dir, SPLIT, 'bounding_box', DISH_ID, f'{DISH_ID}.json')

# Constants for coordinate conversion (used by visualize_bboxes via convert_string_to_bboxes)
BINS = 1000 # This is the default in convert_string_to_bboxes

# --- Load Image ---
img = cv2.imread(rgb_image_path)
if img is None:
    print(f"Erreur : Impossible de charger l'image depuis {rgb_image_path}")
    # Consider raising an error or exiting if the image is essential
    img_display = np.zeros((224, 224, 3), dtype=np.uint8) # Placeholder
    img_h, img_w = 224, 224 # Placeholder
else:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert BGR (OpenCV default) to RGB for Matplotlib
    img_h, img_w = img.shape[:2]
    img_display = img.copy() # Work on a copy

# --- Load Bounding Box JSON ---
try:
    with open(bbox_json_path, 'r') as f:
        bboxes_data = json.load(f)
except FileNotFoundError:
    print(f"Avertissement : Fichier de bounding box non trouvé à {bbox_json_path}. L'image sera affichée sans boîtes.")
    bboxes_data = []
except json.JSONDecodeError:
    print(f"Erreur : Impossible de décoder le JSON depuis {bbox_json_path}. L'image sera affichée sans boîtes.")
    bboxes_data = []

# --- Convert JSON bboxes to string format for visualize_bboxes ---
# The visualize_bboxes function expects a string like: "v0=x1 v1=y1 v2=x2 v3=y2 class_name1 v0=x3 v1=y3 ..."
# The coordinates in the string should be scaled as if they were derived from normalized coordinates multiplied by (BINS - 1).
# Original N5K bbox coordinates are absolute pixel values.
bbox_parts_for_string = []
if bboxes_data and img is not None: # Only proceed if there's data and an image to get dimensions from
    for item in bboxes_data:
        n5k_name = item.get('n5k_name', 'classe_inconnue')
        bbox_coords_abs = item.get('bbox') # Expected: [x_min_abs, y_min_abs, x_max_abs, y_max_abs]

        if bbox_coords_abs and len(bbox_coords_abs) == 4:
            x_min_abs, y_min_abs, x_max_abs, y_max_abs = bbox_coords_abs

            # Convert absolute pixel coordinates to the scaled format expected in the string
            # X_scaled_for_string = (X_absolute / image_dimension_size) * (BINS - 1)
            # Ensure img_w and img_h are not zero to avoid DivisionByZeroError
            if img_w == 0 or img_h == 0:
                print(f"Erreur: Les dimensions de l'image sont nulles (width={img_w}, height={img_h}). Impossible de calculer les coordonnées des boîtes.")
                continue

            v0_val = (x_min_abs / img_w) * (BINS - 1)
            v1_val = (y_min_abs / img_h) * (BINS - 1)
            v2_val = (x_max_abs / img_w) * (BINS - 1)
            v3_val = (y_max_abs / img_h) * (BINS - 1)
            
            # The class name can contain spaces. visualize_bboxes handles this.
            bbox_parts_for_string.append(f"v0={int(round(v0_val))} v1={int(round(v1_val))} v2={int(round(v2_val))} v3={int(round(v3_val))} {n5k_name}")
        else:
            print(f"Avertissement : Données de bbox invalides ou manquantes pour l'item : {item}")

bboxes_str_for_viz = " ".join(bbox_parts_for_string)

# --- Visualize ---
# visualize_bboxes expects an RGB numpy array.
# It returns the image with bounding boxes drawn on it.
if img is not None:
    # Ensure visualize_bboxes is successfully imported
    if 'visualize_bboxes' in globals() or 'visualize_bboxes' in locals():
        # Pass a copy of the image if you don't want the original 'img' array to be modified by visualize_bboxes
        img_with_bboxes = visualize_bboxes(img_display.copy(), bboxes_str_for_viz)

        plt.figure(figsize=(10, 10))
        plt.imshow(img_with_bboxes)
        plt.axis('off')
        plt.title(f"Boîtes Englobantes pour {DISH_ID} (Split: {SPLIT})")
        plt.show()
    else:
        print("Erreur: La fonction 'visualize_bboxes' n'a pas été importée correctement. Visualisation impossible.")
elif img_display is not None : # If image loading failed but we have a placeholder
    plt.figure(figsize=(10, 10))
    plt.imshow(img_display) # Show placeholder or original image if bboxes couldn't be processed
    plt.axis('off')
    plt.title(f"Erreur lors du traitement des boîtes pour {DISH_ID} (Split: {SPLIT})")
    plt.show()
else:
    print("Image non chargée, impossible de visualiser.")
